# Deep Editing

## Importando Librerias

In [1]:

import numpy as np
import os as OperativeSystem
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from numpy import genfromtxt
import shutil
from heapq import nsmallest
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import sklearn
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import OneSidedSelection
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
from keras.models import load_model 
import h5py
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import shuffle

## Definiendo Parametros

In [2]:
directorio="../../"
#directorioPrueba=directorio+"Datasets/txt/test/"
directorioEntrenamientoOut=directorio+"Datasets/txt/sampled_train2/"
directorioEntrenamientoIn=directorio+"Datasets/txt/sampled_train/"
directorioModelosIn=directorio+"modelos/"
directorioPesosIn=directorio+"pesos/"
directorioModelosOut=directorio+"modelos2/"
directorioPesosOut=directorio+"pesos2/"
bases=["Salinas"]
epocas=[500]
lotes=[500]
clases=[17]
bandas=[224]
OS=["","_SMOTE","_ROS"]
E = ["_ENN", "_TL", "_OSS"]
metodosDeEdicionProfunda = {
    "_TL": TomekLinks(sampling_strategy='all', n_jobs=7),
    "_ENN": EditedNearestNeighbours(sampling_strategy='all', n_jobs=7),
    "_OSS": OneSidedSelection(sampling_strategy='all', n_jobs=7)
}

n_capas=["","C1","C2","C3","C4"]


##

In [3]:
inicio= datetime.datetime.now()
contadorBases=0
for base in bases:
    print("\n\n\n")
    print(">>>>> Base: ",base)
    for capa in n_capas:
        print(">>>>>>>>>> Capa: ",capa)
        print("\n\n\n")
        try:

            for os in OS:
                for e in E:

                    print("\n\n")
                    print(">>>Metodo de edicion: ",e," con el metodo de re muestreo:  ",os," para la capa: ",capa, " con la base:  ",base )
                   
                    
                    print("\n")
                    inicio=datetime.datetime.now()        

                    print(">>>  Cargando modelo: ",str(directorioModelosIn+"modelo"+str(capa)+base+""+os+".h5"))    
                    modelo = str(directorioModelosIn+"modelo"+str(capa)+base+""+os+".h5")
                    print(">>>  Cargando pesos: ",  str(directorioPesosIn+"pesos"+str(capa)+base+""+os+".weights.h5")) 
                    pesos = str(directorioPesosIn+"pesos"+str(capa)+base+""+os+".weights.h5")
                    model = tf.keras.models.load_model(modelo)
                    model.load_weights(pesos)

                    #Leer los valores del archivo de entrenamiento 
                    carga=np.loadtxt(str(directorioEntrenamientoIn+str(base)+os+".txt"))
                    #carga=shuffle(carga)
                    datos=carga[:,0:carga.shape[1]-1]
                    clases=carga[:,carga.shape[1]-1]

                    snn_pred = model.predict(datos, batch_size=100)

                    #Se aplica TL a los datos
                    x=snn_pred
                    y=clases
    
                    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>> ",metodosDeEdicionProfunda[e])
                    sampler = metodosDeEdicionProfunda[e]

                    x_res, y_res = sampler.fit_resample(x, y)
                    print(x_res.shape)
                    print(Counter(y_res))
                    #Guardar metodo en archivo
                    indices=sampler.sample_indices_
                    indices.sort()
                    print(">>>indices:",indices)
                    print(">>> forma indices:",indices.shape)
                    
                    Archivo = open(str(directorioEntrenamientoOut+str(base)+str(capa)+""+e+""+os+".txt"),'w')

                    k=0
                    for i in tqdm(range(len(clases))):
                        if i == indices[k]and k< (len(indices)-1):
                            k=k+1
                            for j in range (datos.shape[1]):
                                Archivo.write( f"{datos[i][j]:.15f}"+" ")
                                #Archivo.write( str(int(datos[i][j]))+" ")
                                #print(">>>> Dato: ",str(datos[i][j]))
                            Archivo.write(str(int(clases[i])))
                            #print(">>> Clase: ",str(int(clases[i])))
                            Archivo.write(OperativeSystem.linesep)
                    print(len(y_res),k)
                    Archivo.close()

                    fin=datetime.datetime.now()  
                    print(">>> Tiempo de edicion: ",(fin-inicio))
                    

        except Exception as error:
            print("> Error realizando el proceso para la base: ",base," con el metodo de muestreo: ",capa)
            print(">>Error: ",error)
    contadorBases=contadorBases+1
fin= datetime.datetime.now()
print("\n\n\n")
print(">>> Proceso terminado en : ",(fin-inicio))

    





>>>>> Base:  Salinas
>>>>>>>>>> Capa:  







>>>Metodo de edicion:  _ENN  con el metodo de re muestreo:     para la capa:    con la base:   Salinas


>>>  Cargando modelo:  ../../modelos/modeloSalinas.h5
>>>  Cargando pesos:  ../../pesos/pesosSalinas.weights.h5
778/778 [==============================] - 3s 2ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>>  EditedNearestNeighbours(n_jobs=7, sampling_strategy='all')
(60626, 17)
Counter({0.0: 34669, 8.0: 4494, 9.0: 3689, 15.0: 2285, 6.0: 2159, 2.0: 2142, 7.0: 2111, 10.0: 1791, 5.0: 1304, 16.0: 1070, 12.0: 1011, 1.0: 980, 3.0: 853, 4.0: 739, 11.0: 489, 14.0: 455, 13.0: 385})
>>>indices: [    0     1     2 ... 77766 77769 77770]
>>> forma indices: (60626,)


100%|██████████| 77772/77772 [00:28<00:00, 2687.22it/s]


60626 60625
>>> Tiempo de edicion:  0:00:50.721367



>>>Metodo de edicion:  _TL  con el metodo de re muestreo:     para la capa:    con la base:   Salinas


>>>  Cargando modelo:  ../../modelos/modeloSalinas.h5
>>>  Cargando pesos:  ../../pesos/pesosSalinas.weights.h5
778/778 [==============================] - 2s 2ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>>  TomekLinks(n_jobs=7, sampling_strategy='all')
(74254, 17)
Counter({0.0: 38859, 8.0: 7089, 15.0: 4439, 9.0: 4202, 6.0: 2617, 2.0: 2511, 7.0: 2423, 10.0: 2257, 5.0: 1802, 12.0: 1319, 1.0: 1313, 3.0: 1241, 16.0: 1237, 4.0: 928, 14.0: 725, 11.0: 687, 13.0: 605})
>>>indices: [    0     1     2 ... 77769 77770 77771]
>>> forma indices: (74254,)


100%|██████████| 77772/77772 [00:35<00:00, 2170.70it/s]


74254 74253
>>> Tiempo de edicion:  0:00:52.489448



>>>Metodo de edicion:  _OSS  con el metodo de re muestreo:     para la capa:    con la base:   Salinas


>>>  Cargando modelo:  ../../modelos/modeloSalinas.h5
>>>  Cargando pesos:  ../../pesos/pesosSalinas.weights.h5
778/778 [==============================] - 2s 2ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>>  OneSidedSelection(n_jobs=7, sampling_strategy='all')
(9693, 17)
Counter({0.0: 7483, 8.0: 381, 9.0: 369, 3.0: 351, 10.0: 246, 14.0: 161, 4.0: 151, 11.0: 139, 12.0: 104, 15.0: 98, 5.0: 82, 6.0: 37, 16.0: 36, 2.0: 26, 7.0: 19, 1.0: 9, 13.0: 1})
>>>indices: [    0    22    42 ... 77767 77768 77771]
>>> forma indices: (9693,)


...

670322 670321
>>> Tiempo de edicion:  0:15:35.719868



>>>Metodo de edicion:  _TL  con el metodo de re muestreo:   _ROS  para la capa:  C4  con la base:   Salinas


>>>  Cargando modelo:  ../../modelos/modeloC4Salinas_ROS.h5
>>>  Cargando pesos:  ../../pesos/pesosC4Salinas_ROS.weights.h5
6797/6797 [==============================] - 14s 2ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>>  TomekLinks(n_jobs=7, sampling_strategy='all')
(679660, 20)
Counter({0.0: 39980, 7.0: 39980, 8.0: 39980, 5.0: 39980, 10.0: 39980, 16.0: 39980, 6.0: 39980, 3.0: 39980, 4.0: 39980, 9.0: 39980, 13.0: 39980, 12.0: 39980, 15.0: 39980, 14.0: 39980, 1.0: 39980, 2.0: 39980, 11.0: 39980})
>>>indices: [     0      1      2 ... 679657 679658 679659]
>>> forma indices: (679660,)


100%|██████████| 679660/679660 [05:17<00:00, 2143.67it/s]


679660 679659
>>> Tiempo de edicion:  0:15:35.228331



>>>Metodo de edicion:  _OSS  con el metodo de re muestreo:   _ROS  para la capa:  C4  con la base:   Salinas


>>>  Cargando modelo:  ../../modelos/modeloC4Salinas_ROS.h5
>>>  Cargando pesos:  ../../pesos/pesosC4Salinas_ROS.weights.h5
6797/6797 [==============================] - 13s 2ms/step
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>>>  >>>>>>  OneSidedSelection(n_jobs=7, sampling_strategy='all')
(606505, 20)
Counter({1.0: 39663, 7.0: 39649, 11.0: 39621, 15.0: 39437, 4.0: 39385, 12.0: 39127, 5.0: 39083, 2.0: 38851, 9.0: 38678, 16.0: 38558, 14.0: 38412, 6.0: 38276, 13.0: 37748, 8.0: 37363, 3.0: 36944, 10.0: 25709, 0.0: 1})
>>>indices: [     3      4      6 ... 679657 679658 679659]
>>> forma indices: (606505,)


100%|██████████| 679660/679660 [04:52<00:00, 2321.98it/s]

606505 606504
>>> Tiempo de edicion:  0:14:25.633885




>>> Proceso terminado en :  0:14:25.634886
